In [1]:
import datetime
import os
import pandas as pd
import numpy 
from haversine import haversine
import json
import numpy as np
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
df_all_Store=pd.ExcelFile("./BL 2019Q4 Budget Allocation Model 20191017 JC.xlsx")
df_all_Store=df_all_Store.parse('Sheet3',dtype=str)
print(df_all_Store.shape)
df_all_Store=df_all_Store[['location_id']].drop_duplicates()
print(df_all_Store.shape)

list_all_stores=df_all_Store['location_id'].tolist()

(1420, 4)
(1409, 1)


In [3]:
TA_excel=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
store_ta_zips=TA_excel.parse("view_by_store",dtype=str)

In [4]:
df_existing=store_ta_zips[store_ta_zips['location_id'].isin(list_all_stores)]
store_list_new=[x for x in list_all_stores if x not in df_existing['location_id'].tolist()]
print(store_list_new)
print(len(store_list_new))

['4698', '5394', '5406', '4679', '4708', '5403', '5404', '4704', '5397', '5401', '4692', '5388', '4678', '4702', '4703', '4705', '4706', '5402', '4701', '4707', '4709', '4711', '5398', '5399', '5400', '5408', '5409', '5410', '5411', '5407']
30


In [10]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20191001-134050-499.txt",dtype=str,sep="|")
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.strip().split("-")[0].zfill(5))
store_list=store_list.rename(columns={"zip_cd":"store_zip"})
df_new_with_location=store_list[store_list['location_id'].isin(store_list_new)]
df_new_with_location.shape

(19, 13)

In [11]:
df_new_with_location=df_new_with_location[['location_id','longitude_meas','latitude_meas','state_nm','store_zip']].drop_duplicates()
df_new_with_location['latitude_meas']=df_new_with_location['latitude_meas'].astype(float)
df_new_with_location['longitude_meas']=df_new_with_location['longitude_meas'].astype(float)
df_new_with_location=df_new_with_location[df_new_with_location['latitude_meas']!=0]
df_new_with_location=df_new_with_location[pd.notnull(df_new_with_location['latitude_meas'])]
df_new_with_location.shape

(19, 5)

In [12]:
stores_without_location=[x for x in store_list_new if x not in df_new_with_location['location_id'].tolist()]
stores_without_location

['4701',
 '4707',
 '4709',
 '4711',
 '5398',
 '5399',
 '5400',
 '5408',
 '5409',
 '5410',
 '5411']

In [13]:
dict_new_store_address={}

# 5399 	851 SOUTH 30TH STREET	INDIAN MOUND MALL	HEATH	OH	43056
dict_new_store_address.update({"5399":{"GoogleAddress":"771 S 30th St Ste 9001, Heath, OH 43056","GoogleCoor":[40.0312042,-82.4473616]}})

# 5400 	2235 EAST STATE STREET	HERMITAGE TOWN PLAZA	HERMITAGE	PA	16148-2727
dict_new_store_address.update({"5400":{"GoogleAddress":"2253 E State St, Hermitage, PA 16148","GoogleCoor":[41.234349,-80.4658798]}})

# 4711 	4655 CANAL AVENUE SOUTHWEST	GRANDVILLE MARKETPLACE	GRANDVILLE	MI	49418-8731
dict_new_store_address.update({"4711":{"GoogleAddress":"4485-4655 Canal Ave SW, Grandville, MI 49418","GoogleCoor":[42.8815927,-85.7745291]}})

# 5411 	2318 WEST MERCURY BOULEVARD	TODD'S LANE SHOPS	HAMPTON	VA	23666
dict_new_store_address.update({"5411":{"GoogleAddress":"2318 W Mercury Blvd, Hampton, VA 23666","GoogleCoor":[37.0394621,-76.4073108]}})

# 4709 	5252 BARDSTOWN ROAD	BARDSTOWN PLACE	LOUISVILLE	KY	40291-1766
dict_new_store_address.update({"4709":{"GoogleAddress":"5252 Bardstown Rd, Louisville, KY 40291","GoogleCoor":[38.1680876,-85.6073362]}})

# 5410 	9690 COLERAIN AVENUE	PROSPECT SQUARE	CINCINNATI	OH	45251
dict_new_store_address.update({"5410":{"GoogleAddress":"9690 Colerain Ave, Cincinnati, OH 45251","GoogleCoor":[39.2482448,-84.5948913]}})

# 4707 	14215 SOUTHEAST PETROVITSKY ROAD	FAIRWOOD SQUARE	RENTON	WA	98058-8983
dict_new_store_address.update({"4707":{"GoogleAddress":"14215 SE Petrovitsky Rd, Renton, WA 98058","GoogleCoor":[47.4444178,-122.1519844]}})

# 4701 	5055 W SAHARA AVE	SAHARA DECATUR	LAS VEGAS	NV	89146-3405
dict_new_store_address.update({"4701":{"GoogleAddress":"5055 W Sahara Ave, Las Vegas, NV 89146","GoogleCoor":[36.1436122,-115.2103361]}})

# 5409 	4585 EASTGATE BOULEVARD	TOYS R US CENTER	CINCINNATI	OH	45245-1201
dict_new_store_address.update({"5409":{"GoogleAddress":"4585 Eastgate Blvd, Cincinnati, OH 45245","GoogleCoor":[39.102485,-84.2740534]}})

# 4705 	370 AUTO CENTER CIRCLE	NORTHRIDGE MALL	SALINAS	CA	93907
dict_new_store_address.update({"4705":{"GoogleAddress":"370 Auto Center Cir, Salinas, CA 93906","GoogleCoor":[36.7148622,-121.6607234]}})

# 4706 	3170 CHILI AVE UNIT # T5	CHILI PLAZA	ROCHESTER	NY	14624-5410
dict_new_store_address.update({"4706":{"GoogleAddress":"3170 Chili Ave T5, Rochester, NY 14624","GoogleCoor":[43.1085609,-77.7467213]}})

# 4702 	6375 MONTANA AVENUE	KMART SHOPPING CENTER	EL PASO	TX	79925-2044
dict_new_store_address.update({"4702":{"GoogleAddress":"6375 Montana Ave, El Paso, TX 79925","GoogleCoor":[31.7898616,-106.404997]}})

# 4678 	3501 EAST LINCOLNWAY	CHEYENNE PLAZA	CHEYENNE	WY	82001-6302
dict_new_store_address.update({"4678":{"GoogleAddress":"3501 E Lincolnway, Cheyenne, WY 82001","GoogleCoor":[41.1370675,-104.7674035]}})

# 5407 	14154 EAST WADE HAMPTON BLVD	COLLINS CENTER	GREER	SC	29651-1554
dict_new_store_address.update({"5407":{"GoogleAddress":"14154 E Wade Hampton Blvd, Greer, SC 29651","GoogleCoor":[34.948923,-82.2118793]}})

# 5402 	14603 TELEGRAPH ROAD	TOYS R US CENTER	WOODBRIDGE	VA	22192-4601
dict_new_store_address.update({"5402":{"GoogleAddress":"14603 Telegraph Rd, Woodbridge, VA 22194","GoogleCoor":[38.6373899,-77.2914551]}})

# 5398 	275 ROUTE 18	MIRACLE MALL	EAST BRUNSWICK	NJ	08816-1904
dict_new_store_address.update({"5398":{"GoogleAddress":"275 NJ-18, East Brunswick, NJ 08816","GoogleCoor":[40.4592229,-74.4049486]}})

# 5408 	3420 SOUTHWEST DURHAM DRIVE	PATTERSON PLACE	DURHAM	NC	27707-3351
dict_new_store_address.update({"5408":{"GoogleAddress":"3420 SW Durham Dr, Durham, NC 27707","GoogleCoor":[35.9516434,-78.9887268]}})

In [14]:
df_new_google_location=pd.DataFrame({"location_id":stores_without_location},index=[x for x in range(len(stores_without_location))])
df_new_google_location['longitude_meas']=df_new_google_location['location_id'].apply(lambda x: dict_new_store_address[x]['GoogleCoor'][1])
df_new_google_location['latitude_meas']=df_new_google_location['location_id'].apply(lambda x: dict_new_store_address[x]['GoogleCoor'][0])
df_new_google_location['store_zip']=df_new_google_location['location_id'].apply(lambda x: dict_new_store_address[x]['GoogleAddress'][-5:])
df_new_google_location['state_nm']=df_new_google_location['location_id'].apply(lambda x: dict_new_store_address[x]['GoogleAddress'].split(", ")[-1][:2])

df_new_google_location.shape

(11, 5)

In [20]:
df_new_google_location.head(2)

,location_id,longitude_meas,latitude_meas,store_zip,state_nm
0,4701,-115.210336,36.143612,89146,NV
1,4707,-122.151984,47.444418,98058,WA


In [21]:
df_new_with_location.head(2)

,location_id,longitude_meas,latitude_meas,state_nm,store_zip
1008,4678,-104.767025,41.137076,WY,82001
1015,4692,-122.495657,48.775269,WA,98225


In [22]:
df_new_store_10_miles=df_new_google_location.append(df_new_with_location)
df_new_store_10_miles.shape

(30, 5)

In [16]:
df_output_existing=store_list[['location_id','state_nm','store_zip','latitude_meas','longitude_meas']]
df_output_existing=df_output_existing[df_output_existing['location_id'].isin(df_existing['location_id'].unique().tolist())]
df_zip_cd=pd.DataFrame()
df_existing=df_existing.reset_index()
del df_existing['index']
for ind,row in df_existing.iterrows():
    store_num=row['location_id']
    zip_list_P=eval(row['trans_P_zips_70_within_TA'])
    zip_list_S=eval(row['trans_S_zips_70_within_TA'])
    zip_list_10=eval(row['zips_in_10'])
    
    df_P=pd.DataFrame({"zip":zip_list_P},index=["P"]*len(zip_list_P))
    df_S=pd.DataFrame({"zip":zip_list_S},index=["S"]*len(zip_list_S))
    df_10=pd.DataFrame({"zip":zip_list_10},index=["zip_in_10"]*len(zip_list_10))
    
    df=pd.concat([df_P,df_S,df_10]).reset_index().rename(columns={"index":"trans_flag"})
    df['location_id']=store_num
    df_zip_cd=df_zip_cd.append(df)
df_output_existing=pd.merge(df_output_existing,df_zip_cd,on="location_id",how="left")

In [17]:
print(df_output_existing.shape)
df_output_existing.head(2)

(22728, 7)


,location_id,state_nm,store_zip,latitude_meas,longitude_meas,trans_flag,zip
0,3,GA,30906,33.42157,-82.018921,P,30906
1,3,GA,30906,33.42157,-82.018921,S,30904


In [18]:
df_output_existing['Distance (mile)']=np.nan
df_output_existing=df_output_existing.reset_index()
del df_output_existing['index']

for ind, row in df_output_existing.iterrows():
    dist=haversine((row['latitude_meas'],row['longitude_meas']),zip_centers[row['zip']],miles=True)
    df_output_existing.loc[ind,"Distance (mile)"]=dist
    

In [54]:
df_output_existing[df_output_existing['location_id']=="1100"]

,location_id,state_nm,store_zip,latitude_meas,longitude_meas,trans_flag,zip,Distance (mile)


In [44]:
df_output_new=pd.DataFrame()
for ind,row in df_new_store_10_miles.iterrows():
    location_id=row['location_id']
    latitude_meas=row['latitude_meas']
    longitude_meas=row['longitude_meas']
    
    df=pd.DataFrame()
    for zip_cd,v in zip_centers.items():
        dist=haversine((latitude_meas,longitude_meas),v,miles=True)
        if dist<=10:
            df=df.append(pd.DataFrame({"zip":zip_cd,"Distance (mile)":dist},index=["zip_in_10"]))
    df=df.reset_index().rename(columns={"index":"trans_flag"})
    df['location_id']=location_id
    df_output_new=df_output_new.append(df)
df_output_new=pd.merge(df_new_store_10_miles,df_output_new,on="location_id",how="left")
df_output_new.shape

(764, 8)

In [46]:
df_output_all=df_output_new.append(df_output_existing)
print(df_output_all.shape,df_output_all['location_id'].nunique())

(23492, 8) 1407


In [47]:
[x for x in list_all_stores if x not in df_output_all['location_id'].tolist()]

# The 2 stores look closed

['1202', '1100']

In [59]:
df_output_all=df_output_all[['location_id','trans_flag','zip','state_nm','store_zip','latitude_meas','longitude_meas','Distance (mile)']]


In [62]:
writer=pd.ExcelWriter("./BL_all_store_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_all.to_excel(writer,"all_PS10_zips_BL",index=False)
writer.save()